#Create Vector Indexes

###Mosaic AI Vector Search and Vector Indexes
Mosaic AI Vector Search is a vector database that is built into the Databricks Data Intelligence Platform and integrated with its governance and productivity tools. A vector database is a database that is optimized to store and retrieve embeddings. Embeddings are mathematical representations of the semantic content of data, typically text or image data. [Read More](https://docs.databricks.com/en/generative-ai/vector-search.html)

### Vector Indexes
Let us start creating vector indexes

<img src="https://raw.githubusercontent.com/databricks-industry-solutions/CareCost-Compass/refs/heads/main/resources/build_3.png" alt="Vector Indexes" width="900"/>

We will be creating two vector indexes for this project.
1. Vector Index for the parsed Summary of Benefits and Coverage chunks
2. Vector Index for CPT codes and descriptions


#####Install libraries and import utility methods

In [0]:
%pip install -qqqq -U mlflow databricks-feature-engineering langchain databricks-langchain

In [0]:
dbutils.library.restartPython()

In [0]:
%run ./utils/utils

####Create a Vector Search endpoint
vector Search Endpoint serves the vector search index. You can query and update the endpoint using the REST API or the SDK. Endpoints scale automatically to support the size of the index or the number of concurrent requests. See [Create a vector search endpoint](https://docs.databricks.com/en/generative-ai/create-query-vector-search.html#create-a-vector-search-endpoint) for instructions.

In [0]:
# name for the vector search endpoint
vector_search_endpoint_name = "care_cost_vs_endpoint"

# We are using an embedding endpoint available in Databricks Workspace
# If needed we can use custom embedding endpoints as well
embedding_endpoint_name = "databricks-bge-large-en"

# Define the source tables, index name and key fields
sbc_source_data_table = f"{catalog}.{schema}.{sbc_details_table_name}"
sbc_source_data_table_id_field = "id"
sbc_source_data_table_text_field = "content"
sbc_vector_index_name = f"{sbc_source_data_table}_index"

cpt_source_data_table = f"{catalog}.{schema}.{cpt_code_table_name}"
cpt_source_data_table_id_field = "id"
cpt_source_data_table_text_field = "description"
cpt_vector_index_name = f"{cpt_source_data_table}_index"

**NOTE:** Below command creates a Vector Search Endpoint and will take few minutes to complete. 

In [0]:
from databricks.vector_search.client import VectorSearchClient
from datetime import timedelta
import time

# create the vector search endpoint if it does not exist
# same endpoint can be used to serve both the indexes
vsc = VectorSearchClient(disable_notice=True)

try:
    vsc.create_endpoint(name=vector_search_endpoint_name, endpoint_type="STANDARD")

    time.sleep(5)

    vsc.wait_for_endpoint(
        name=vector_search_endpoint_name, timeout=timedelta(minutes=60), verbose=True
    )

    print(f"Endpoint named {vector_search_endpoint_name} is ready.")

except Exception as e:
    if "already exists" in str(e):
        print(f"Endpoint named {vector_search_endpoint_name} already exists.")
    else:
        raise e

#### Check if embeddings endpoint exists

We will use the existing `databricks-bge-large-en` endpoint for embeddings

In [0]:
import mlflow
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")

In [0]:
[ep for ep in client.list_endpoints() if ep["name"] == embedding_endpoint_name]

#### Test the embeddings endpoint

In [0]:
client.predict(
    endpoint="databricks-bge-large-en", inputs={"input": ["What is Apache Spark?"]}
)

####Create Vector Search Index
The vector search index is created from a Delta table and is optimized to provide real-time approximate nearest neighbor searches. The goal of the search is to identify documents that are similar to the query. Vector search indexes appear in and are governed by Unity Catalog. To learn more about creating Vector Indexes, visit this [link](https://docs.databricks.com/en/generative-ai/create-query-vector-search.html). 

We will now create the vector indexes. Our vector index will be of `Delta Sync Index` type. [[Read More](https://docs.databricks.com/en/generative-ai/create-query-vector-search.html#create-a-vector-search-index)] 

We will use a Sync Mode of `TRIGGERED` as our table updates are not happening frequently and sync latency is not an issue for us. [[Read More](https://docs.databricks.com/en/generative-ai/create-query-vector-search.html#create-a-vector-search-index:~:text=embedding%20table.-,Sync%20mode%3A,-Continuous%20keeps%20the)]

**NOTE:** In order for vector search to automatically sync updates, we need to enable ChangeDataFeed on the source table.

#####Create CPT Code Vector Index

**NOTE:** Below section creates a vector search index and does an initial sync. Some time this could take longer and the cell execution might timeout. You can re-run the cell to finish to completion

In [0]:
try:
    cpt_index = vsc.create_delta_sync_index_and_wait(
        endpoint_name=vector_search_endpoint_name,
        index_name=cpt_vector_index_name,
        source_table_name=cpt_source_data_table,
        primary_key=cpt_source_data_table_id_field,
        embedding_source_column=cpt_source_data_table_text_field,
        embedding_model_endpoint_name=embedding_endpoint_name,
        pipeline_type="TRIGGERED",
        verbose=True,
    )
except Exception as e:
    if "already exists" in str(e):
        print(f"Index named {vector_search_endpoint_name} already exists.")
        cpt_index = vsc.get_index(vector_search_endpoint_name, cpt_vector_index_name)
    else:
        raise e

#### Quick Test of Indexes

In [0]:
%pip install -qqqq -U databricks-vectorsearch 
dbutils.library.restartPython()

In [0]:
%run ./utils/utils

In [0]:
# name for the vector search endpoint
vector_search_endpoint_name = "care_cost_vs_endpoint"

# We are using an embedding endpoint available in Databricks Workspace
# If needed we can use custom embedding endpoints as well
embedding_endpoint_name = "databricks-bge-large-en"

# Define the source tables, index name and key fields
sbc_source_data_table = f"{catalog}.{schema}.{sbc_details_table_name}"
sbc_source_data_table_id_field = "id"
sbc_source_data_table_text_field = "content"
sbc_vector_index_name = f"{sbc_source_data_table}_index"

cpt_source_data_table = f"{catalog}.{schema}.{cpt_code_table_name}"
cpt_source_data_table_id_field = "id"
cpt_source_data_table_text_field = "description"
cpt_vector_index_name = f"{cpt_source_data_table}_index"

In [0]:
from databricks.vector_search.client import VectorSearchClient
from datetime import timedelta
import time

# create the vector search endpoint if it does not exist
# same endpoint can be used to serve both the indexes
vsc = VectorSearchClient(disable_notice=True)

In [0]:
cpt_index = vsc.get_index(vector_search_endpoint_name, cpt_vector_index_name)

In [0]:
results = cpt_index.similarity_search(
    query_text="How much does Xray of shoulder cost?",
    columns=["id", "code", "description"],
    num_results=3,
)

if results["result"]["row_count"] > 0:
    display(results["result"]["data_array"])
else:
    print("No records")